This Notebook is the implementation of fine-tuning Mistral LLM on FinQA Dataset for Question-answering task. Evaluation was performed on test dataset and results have been reported

### nstall Required packages and libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.34.0 trl==0.4.7
!pip install huggingface_hub
!pip install -q datasets

import numpy as np
import re
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
import pandas as pd
import random
from datasets import Dataset
from datasets import load_dataset
import pandas as pd
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 9.3 MB/s eta 0:00:00


### Login into hugging face hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Load the data

- Train and Test data has been loaded
- FINQA has separate Split for train and test data and hence same has been loaded from hugging face hub
- Data has been converted into a format which can be used to fine tune the LLMs

In [ ]:

pd.set_option('display.max_colwidth', -1)
dataset = load_dataset("dreamerdeo/finqa",split="train") ### Load train data from hugging face hub

dataset_pd = dataset.to_pandas() ### convert to pandas
 
def table_format(x_input): ### format the tabular data in the financial report in pipe delimited format
  fin_list = []
  x = x_input.tolist()
  for i in range(len(x)):
    j = x[i]
    x_str="|".join(str(element) for element in j)
    fin_list.append(x_str)
  final_str = f"\n".join(str(element) for element in fin_list)
  return final_str


def conv_list_string(x): ### Convert the list of sentences into a single paragraph of sentences
  x_list = x.tolist()
  x_string="".join(str(element) for element in x)
  return x_string

dataset_pd['pre_text'] = dataset_pd['pre_text'].map(conv_list_string)
dataset_pd['table'] = dataset_pd['table'].apply(table_format)
dataset_pd['post_text'] = dataset_pd['post_text'].map(conv_list_string)
dataset_pd['gold_evidence'] = dataset_pd['gold_evidence'].map(conv_list_string)


 ### Instructions to be overlaid to the train dataset
instructions = [
    "Financial data and expert analysis has been provided as context, use the provided context to answer the question.",
    "Please provide the required answer to this expert-authored finance question based on the context given.",
    "A deep financial question based on a financial report has been given along with required context, could you help answer the financial question.",
    "Analysis based on financial documents has been given as context, please answer the question using the context.",
    "Financial report data and analysis has been given as context , please respond to the question using the context given."
]

ins_list = []
for ins in range(6251):
  ins = random.choice(instructions)
  ins_list.append(ins)

## Add the instructions
dataset_pd['instructions'] = ins_list

ins_str = 'context has financial text followed by pipe delimited table and some more financial text is given below the pipe delimited table. Use multi-step numerical reasoning and logical reasoning wherever required to answer the given question'
dataset_pd['final_instruction'] = f'###Instruction:'  + dataset_pd['instructions'] + ins_str + f'\n\n' + f'####context:' + f'\n' + dataset_pd['pre_text'] + f'\n'+ dataset_pd['table'] + f'\n' + dataset_pd['post_text'] + f'\n\n' +f'####question:' + dataset_pd['question'] +  f'\n\n###Answer:' +  dataset_pd['answer']

from datasets import Dataset
train_df = Dataset.from_pandas(dataset_pd)
dataset_dct = train_df.train_test_split(test_size=0.25)  ### Create validation dataset from train datasetb

### Get the Train and Test Dataset from Hugging Face Dataset Dict object
df_train = dataset_dct["train"]
df_eval = dataset_dct["test"]

<ipython-input-2-222c4df8f7fc>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
Repo card metadata block was not found. Setting CardData to empty.


### Fine-Tuning Framework Parameters

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"



# Fine-tuned model name
new_model = "mistral-7b-finqa-rank-64-vj"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64 ### RANK PARAMETER

# Alpha parameter for LoRA scaling
lora_alpha = 128

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2000

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Perform Fine-tuning

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load  tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj"]
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=df_train,
    eval_dataset=df_eval,
    peft_config=peft_config,
    dataset_text_field="final_instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/4688 [00:00<?, ? examples/s]

Map:   0%|          | 0/1563 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.845500,0.841948


In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"

# Fine-tuned model name
new_model = "mistral-7b-finqa-rank-64-vj"


# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Push the fine-tuned model to Hugging Face hub and save it

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vijayshankar245/mistral-7b-finqa-rank-64-vj/commit/7dd1179350891cd1ad2b8eb298fd0c32202a1a80', commit_message='Upload tokenizer', commit_description='', oid='7dd1179350891cd1ad2b8eb298fd0c32202a1a80', pr_url=None, pr_revision=None, pr_num=None)

### Evaluation

Apply the necessary formatting on test data (Same steps which were followed for train dataset)

In [ ]:
dataset = load_dataset("dreamerdeo/finqa",split="test")

dataset_pd = dataset.to_pandas()

def table_format(x_input):
  fin_list = []
  x = x_input.tolist()
  for i in range(len(x)):
    j = x[i]
    x_str="|".join(str(element) for element in j)
    fin_list.append(x_str)
  final_str = f"\n".join(str(element) for element in fin_list)
  return final_str


def conv_list_string(x):
  x_list = x.tolist()
  x_string="".join(str(element) for element in x)
  return x_string

dataset_pd['pre_text'] = dataset_pd['pre_text'].map(conv_list_string)
dataset_pd['table'] = dataset_pd['table'].apply(table_format)
dataset_pd['post_text'] = dataset_pd['post_text'].map(conv_list_string)
dataset_pd['gold_evidence'] = dataset_pd['gold_evidence'].map(conv_list_string)





ins_str = 'context has financial text followed by pipe delimited table and some more financial text is given below the pipe delimited table. Use multi-step numerical reasoning and logical reasoning wherever required to answer the given question'
dataset_pd['final_instruction'] = ins_str + f'\n\n' + f'####context:' + f'\n' + dataset_pd['pre_text'] + f'\n'+ dataset_pd['table'] + f'\n' + dataset_pd['post_text'] + f'\n\n' +f'####question:' + dataset_pd['question']

from datasets import Dataset
test_df = Dataset.from_pandas(dataset_pd)

Repo card metadata block was not found. Setting CardData to empty.


### Load the fine-tuned model for evaluation

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
model = "vijayshankar245/mistral-7b-finqa-rank-64-vj"
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# Run text generation pipeline with our next model

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,return_full_text=False,temperature=0.1,top_p =0.9,max_new_tokens = 15)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Perform the evaluation and extract the results

In [ ]:
import time
start_time = time.time()
from transformers.pipelines.pt_utils import KeyDataset
output_list = []
j=0
for i in pipe(KeyDataset(test_df, "final_instruction")):
  j=j+1
  print(i[0]['generated_text'])
  output_list.append(i[0]['generated_text'])
  print(j)
  df = pd.DataFrame({'output': output_list})
print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




### Answers:94 million dollars, 1.
1


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




###Answer:14.5% of total facilities are
2


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


...



###Answer:10.675%


1145


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 issued as consideration?

###Answer:27.81
1146


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 in millions

###Answer:3.4billion\\n
1147
--- 46548.22018003464 seconds ---


In [ ]:
dataset_pd['prediction'] = output_list ### Add the prediction output to test dataset

### Post Processing

In [ ]:
def map_yes_no(x):
    x = str(x)
    if x.lower() in 'yes':
        return '1'
    elif x.lower() in 'no':
        return '0'
    else:
        return x


def remove_non_numeric(x):
    x = str(x)
    pattern = re.compile(r"[^\d.]")
    x = pattern.sub("", x)
    return x

def other_formatting(x):
  x =str(x)
  x=x.replace('...', '')
  x=x.replace('..', '')
  tofind = '.'
  if x.count(tofind) > 1:
    pos = x.index(tofind, x.index(tofind) + 1)
    x = x[:pos] + '' + x[pos + 1:]
    return x
  else:
    return x


def replace_blanks(x):
  x =str(x)
  if x.strip() == '':
    return '0'
  else:
    return x

In [ ]:

dataset_pd['formatted_answer']  = dataset_pd['answer'].apply(map_yes_no)
dataset_pd['formatted_answer'] = dataset_pd['formatted_answer'].apply(remove_non_numeric)
dataset_pd['formatted_answer'] = dataset_pd['formatted_answer'].apply(other_formatting)
dataset_pd['formatted_answer'] = dataset_pd['formatted_answer'].apply(replace_blanks)
dataset_pd['formatted_answer'] = dataset_pd['formatted_answer'].astype(float)
dataset_pd['formatted_answer'] = dataset_pd['formatted_answer'].round()

dataset_pd['formatted_prediction']  = dataset_pd['prediction'].apply(map_yes_no)
dataset_pd['formatted_prediction'] = dataset_pd['formatted_prediction'].apply(remove_non_numeric)
dataset_pd['formatted_prediction'] = dataset_pd['formatted_prediction'].apply(other_formatting)
dataset_pd['formatted_prediction'] = dataset_pd['formatted_prediction'].apply(replace_blanks)
dataset_pd['formatted_prediction'] = dataset_pd['formatted_prediction'].apply(other_formatting)
dataset_pd['formatted_prediction'] = dataset_pd['formatted_prediction'].astype(float)
dataset_pd['formatted_prediction'] = dataset_pd['formatted_prediction'].round()

### Final Score

In [ ]:
import numpy as np
dataset_pd['match_ind'] = np.where( dataset_pd['formatted_answer'] == dataset_pd['formatted_prediction'], 1,0)
dataset_pd['match_ind'].mean()

0.32868352223190933

### The exact match accuracy score for Mistral model  is 0.33